In [ ]:
#####################################################################################
# Audio-driven upper-body motion synthesis on a humanoid robot
# Computer Science Tripos Part III Project
# Jan Ondras (jo356@cam.ac.uk), Trinity College, University of Cambridge
# 2017/18
#####################################################################################
# Training, validation and testing of the LSTM-SI model
#####################################################################################

In [ ]:
###############################################################################################################
# LSTM Training and testing
# PER SUBJECT
###############################################################################################################
# Load segmented data, already split
# NOT Rescale target angles to range [0,1] (already done when segmenting)
###############################################################################################################

import numpy as np
import glob
import os
import time
from keras.models import Sequential
from keras.layers import Dense, LSTM, Masking, TimeDistributed, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

np.random.seed(37) # for reproducibility

AF_type = 'AF_logFB26_norm'
SEG_folder = 'Segments_logFB26'
TE_folder = 'TrainingExamples_16kHz'
unique_srt_VIDs = np.load('./../Dataset/'+TE_folder+'/te_unique_srt_VIDs.npz')['unique_srt_VIDs'] # sorted VIDs
all_srt_VIDs = np.load('./../Dataset/'+TE_folder+'/te_VIDs.npz')['VIDs']
unique_srt_SIDs = np.array([x[:5] for i, x in enumerate(unique_srt_VIDs) if i % 2 == 0]) # ['PID02', 'PID05', ..

# Best LSTM architecture from LSTM SI; no dropout
N_runs = 1
SEGMENT_LEN = 300 # == X_train.shape[1]
N_epochs = 500                    # this value is saved as N_epochs
# train_batch_size = 15000 # same for all models; for LOGFB26 and also for MFCC13
N_features = 26
N_targets  = 11
N_LSTM_units = 12 # BEST FOUND in LSTM_SI
useDropout = False

model_checkpoint_path_prefix = './ModelCheckpoints/LSTM_SD/'
save_training_hist_path_prefix = './../Dataset/'+TE_folder+'/Results/LSTM_SD/trainHistES_' # ES = early stop

# Train model for each subject
st = time.time()
for s, SID in enumerate(unique_srt_SIDs):

    print SID

    ######################################################################################################
    # Load segmented data, already split
    ds = np.load('./../Dataset/'+TE_folder+'/' + SEG_folder + '/perSID/seg_' + SID + '.npz')

    X_train = ds['X_train'] 
    Y_train = ds['Y_train'] 

    X_val = ds['X_val'] 
    Y_val = ds['Y_val'] 

    X_val_RT = ds['X_val_RT'] 
    Y_val_RT = ds['Y_val_RT'] 

    X_test = ds['X_test'] 
    Y_test = ds['Y_test'] 

    X_test_RT = ds['X_test_RT'] 
    Y_test_RT = ds['Y_test_RT'] 

    N_train_seg =    ds['N_train_seg']
    N_val_seg =      ds['N_val_seg']
    N_val_RT_seg =   ds['N_val_RT_seg']
    N_test_seg =     ds['N_test_seg']
    N_test_RT_seg =  ds['N_test_RT_seg']

    ######################################################################################################
    # TRAIN & VALIDATE: save best model and training history
    # Sequence tagging approach (LSTMs)
    # if all feature values at a timestep are equal to mask_value=0., then the timestep is skipped

    train_batch_size = X_train.shape[0] # for N_LSTM_units = 128 and less
    print("\tTrain batch size: {:d}".format(train_batch_size))

    # SET MODEL TYPE
    model_type = '{:d}_{:d}_{:02d}_{:}'.format(N_runs, N_features, N_LSTM_units, SID)
    print "\tMODEL TYPE:\t\t", model_type
    if not os.path.isdir(model_checkpoint_path_prefix + model_type):
        os.mkdir(model_checkpoint_path_prefix + model_type)

    # Create LSTM model
    model = Sequential()
    model.add( Masking(mask_value=0., input_shape=(SEGMENT_LEN, N_features)) )
    if useDropout:
        model.add( Dropout(dropouts[0]) )
    model.add( LSTM(N_LSTM_units, return_sequences=True) )
    if useDropout:
        model.add( Dropout(dropouts[1]) )    
    model.add( TimeDistributed(Dense(N_targets, activation='sigmoid')) )
    model.compile(loss='mean_squared_error', optimizer='adam')
    #print model.summary()

    # Set early stopping
    early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

    # Checkpoint model weights and the model itself: at each epoch
    model_checkpoint_name = 'm_{epoch:04d}_{loss:.4f}_{val_loss:.4f}.hdf5'
    model_checkpoint = ModelCheckpoint(model_checkpoint_path_prefix + model_type + '/' + model_checkpoint_name, monitor='val_loss', verbose=0, 
                                       save_best_only=True, save_weights_only=False, mode='auto', period=1)

    loss = []
    val_loss = []
    for i in range(N_runs):
        # Tain & validate
        hist = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=N_epochs, shuffle=True,  
                         batch_size=train_batch_size, verbose=0, 
                         callbacks=[early_stop, model_checkpoint]
                        )
        loss.append( hist.history['loss'] )
        val_loss.append( hist.history['val_loss'] )
    # Save training history
    np.savez(save_training_hist_path_prefix + model_type + '.npz', 
            loss=loss, val_loss=val_loss, 
            N_params=model.count_params(), train_batch_size=train_batch_size, 
            N_runs=N_runs, N_LSTM_units=N_LSTM_units, N_epochs=N_epochs)
    print "\tEarly stop at epoch:", len(hist.history['loss'])
    print "\tSaved training history: ", save_training_hist_path_prefix + model_type + '.npz'
    print "\tBest model saved to: ", model_checkpoint_path_prefix + model_type + '/' + model_checkpoint_name
    print "\tTime taken: ", time.time()-st, (time.time()-st)/60.  
    
# All: Time taken:  3285.47157311 54.7578600009
# all 250 min, for 500 epochs

In [ ]:
######################################################################################################
# Evaluate on VALIDATION & TEST SET;
# save results
# DONE
######################################################################################################
from postprocessingutils import save_predictions_and_eval
from keras.models import load_model
import numpy as np
import glob
import os
import time

np.random.seed(37) # for reproducibility

AF_type = 'AF_logFB26_norm'
SEG_folder = 'Segments_logFB26'
TE_folder = 'TrainingExamples_16kHz'
unique_srt_VIDs = np.load('./../Dataset/'+TE_folder+'/te_unique_srt_VIDs.npz')['unique_srt_VIDs'] # sorted VIDs
all_srt_VIDs = np.load('./../Dataset/'+TE_folder+'/te_VIDs.npz')['VIDs']
unique_srt_SIDs = np.array([x[:5] for i, x in enumerate(unique_srt_VIDs) if i % 2 == 0]) # ['PID02', 'PID05', ..

# Best LSTM architecture from LSTM SI; no dropout
N_runs = 1
SEGMENT_LEN = 300 # == X_train.shape[1]
N_features = 26
N_targets  = 11
N_LSTM_units = 12 # BEST FOUND in LSTM_SI

save_results_path_prefix = './../Dataset/'+TE_folder+'/Results/LSTM_SD/'
model_checkpoint_path_prefix = './ModelCheckpoints/LSTM_SD/{:d}_{:d}_{:d}_'.format(N_runs, N_features, N_LSTM_units)

# Evaluate for each subject
st = time.time()
for s, SID in enumerate(unique_srt_SIDs):

    print SID
    
    #########################################
    # Load trained model for this SID
    test_model_name = sorted(glob.glob(model_checkpoint_path_prefix + SID + '/m_*'))[-1]
    N_epochs = int( (test_model_name.split('/')[-1]).split('_')[1] )
    print "\t Loaded model:", test_model_name
    print "\t # epochs:", N_epochs
    model = load_model( test_model_name )
    
    ######################################################################################################
    # Load segmented data, already split
    ds = np.load('./../Dataset/'+TE_folder+'/' + SEG_folder + '/perSID/seg_' + SID + '.npz')

    X_val_RT = ds['X_val_RT'] 
    Y_val_RT = ds['Y_val_RT'] 

    X_test_RT = ds['X_test_RT'] 
    Y_test_RT = ds['Y_test_RT'] 

    N_val_RT_seg =   ds['N_val_RT_seg']
    N_test_RT_seg =  ds['N_test_RT_seg']
    
    #######################
    # Load the dataset split (for this SID); concat from both VIDs
    ds1 = np.load('./../Dataset/'+TE_folder+'/Dataset_split/split_masks_' + SID  + 'Task2.npz')
    ds2 = np.load('./../Dataset/'+TE_folder+'/Dataset_split/split_masks_' + SID  + 'Task3.npz')

    ###############################################################################################################
    # Evaluate on validation set
    # ONLINE TESTING (as if new timesteps arrive one-by-one)
    
    val_batch_size = X_val_RT.shape[0]
    Y_val_pred = model.predict(X_val_RT, batch_size=val_batch_size, verbose=1)  
    Y_val_pred = Y_val_pred[:, -1, :] # last item from each segment is the (ONLINE) final prediction
    Y_val_true = Y_val_RT[:, -1, :]
    X_val_RT_last = X_val_RT[:, -1, :]
    print X_val_RT.shape, X_val_RT_last.shape, Y_val_pred.shape, Y_val_true.shape
    
    # Save results: predictions will be saved in radians; for generation on robot
    # Raw and smoothed (low-pass 4Hz)
    val_VIDs = [SID + 'Task2', SID + 'Task3']
    SD_offsets = [len(ds1['train_mask']), len(ds2['train_mask']) ]
    
    if N_val_RT_seg[0] != len(ds1['val_mask']) or N_val_RT_seg[1] != len(ds2['val_mask']):
        raise ValueError('PROBLEM!!!')
    
    save_predictions_and_eval(save_results_path_prefix + 'MSBMvaltest_{:d}_{:d}_{:d}_{:}'.format(N_runs, N_features, N_LSTM_units, SID), 
                     X_val_RT_last, Y_val_true, Y_val_pred, 'LSTM_SD', SEGMENT_LEN, val_VIDs, N_val_RT_seg, 
                            SD_offsets, 
                             N_params=model.count_params(), N_epochs=N_epochs  )
    
    ###############################################################################################################
    # Evaluate on testing set: 
    # ONLINE TESTING (as if new timesteps arrive one-by-one)
    
    test_batch_size = X_test_RT.shape[0]
    Y_test_pred = model.predict(X_test_RT, batch_size=test_batch_size, verbose=1)
    Y_test_pred = Y_test_pred[:, -1, :] # last item from each segment is the (ONLINE) final prediction
    Y_test_true = Y_test_RT[:, -1, :]
    X_test_RT_last = X_test_RT[:, -1, :]
    print X_test_RT.shape, X_test_RT_last.shape, Y_test_pred.shape, Y_test_true.shape

    # Save results: predictions will be saved in radians; for generation on robot
    # Raw and smoothed (low-pass 4Hz)
    test_VIDs = [SID + 'Task2', SID + 'Task3']
    SD_offsets = [len(ds1['train_mask']) + len(ds1['val_mask']), len(ds2['train_mask']) + len(ds2['val_mask'])]
    save_predictions_and_eval(save_results_path_prefix + 'MSBMtest_{:d}_{:d}_{:d}_{:}'.format(N_runs, N_features, N_LSTM_units, SID), 
                     X_test_RT_last, Y_test_true, Y_test_pred, 'LSTM_SD', SEGMENT_LEN, test_VIDs, N_test_RT_seg, 
                              SD_offsets,
                              N_params=model.count_params(), N_epochs=N_epochs )       
    
    print "\tTime taken: ", time.time()-st, (time.time()-st)/60.  

In [ ]:
# Copy the best model for each subject
# DONE

from shutil import copy2

model_checkpoint_path_prefix = './ModelCheckpoints/LSTM_SD/{:d}_{:d}_{:d}_'.format(N_runs, N_features, N_LSTM_units)

for s, SID in enumerate(unique_srt_SIDs):
    test_model_name = sorted(glob.glob(model_checkpoint_path_prefix + SID + '/m_*'))[-1]   
    new_name = model_checkpoint_path_prefix + SID + '_' + test_model_name.split('/')[-1]
    copy2(test_model_name, new_name)
    print new_name
